In [1]:
(require '[clojupyter.misc.helper :as helper])
(run! helper/add-dependencies '[[sicmutils "0.18.0"]])

(ns chp6
    (:refer-clojure :exclude [partial zero? + - * / ref compare =])
    (:require [sicmutils.env :refer :all]
              [sicmutils.value :as v]))
(def render (comp ->infix simplify))

#'chp6/render

## 6.1 Vector fields Over a map

### Restricted Vector Fields
- $\mu$: a map from manifold $\mathsf{N}$ to manifold $\mathsf{M}$
- $\mathsf{f}$: a manifold function on $\mathsf{M}$
- $\mathsf{u}$: a vector field on $\mathsf{M}$
- Then $\mathsf{u}_{\mu}$ is a restricted vector field over the map $\mu$
- $\mathsf{u}_{\mu}(\mathsf{f}) (\mathsf{n}) = \mathsf{u}(\mathsf{f}) (\mu(\mathsf{n}))$
- Note that $\mathsf{u}_{\mu}(\mathsf{f})$ is a manifold function on $\mathsf{N}$. It is also referred to as a vector over the map $\mu$

In [2]:
vector-field->vector-field-over-map

#function[sicmutils.calculus.map/vector-field->vector-field-over-map]

### Differential of a Map
- $\mu$: a map from manifold $\mathsf{N}$ to manifold $\mathsf{M}$
- $\mathsf{f}$: a manifold function on $\mathsf{M}$
- $\mathsf{v}$: a vector field on $\mathsf{N}$
- the differential of a map takes three arguments:
- 
$d\mu(\mathsf{v})(\mathsf{f})(\mathsf{n}) = \mathsf{v}(\mathsf{f}\circ \mu) (\mathsf{n})$
- the differential of a map $\mu$ applied to a vector field $\mathsf{v}$ on $\mathsf{N}$ is a vector field over the map $\mu$.
- the differential of the map transport a vector field from the source manifold $\mathsf{N}$ to the target manifold $\mathsf{M}$

In [3]:
differential

#function[sicmutils.calculus.map/differential-of-map]

### Velocity at a Time
- $\mu$: a map from the time line to manifold $\mathsf{M}$
- $\partial/\partial \mathsf{t}$: the basis vector on the time line (this is a vector field from the source manifold)
- $d\mu (\partial/\partial \mathsf{t})$: the vector field over the map $\mu$ that compute the rate of change of functions on $\mathsf{M}$ along the path that is the image of $\mu$. Thus, $d\mu (\partial/\partial \mathsf{t})$ gives the velocity vector at each moment
- This solves the issue where a path crosses itself and leads to multiple vectors at a point on the manifold $\mathsf{M}$

## 6.2 One-Form Fields Over a Map

- $\mu$: a map from manifold $\mathsf{N}$ to manifold $\mathsf{M}$
- $\mathsf{f}$: a manifold function on $\mathsf{M}$
- $\mathsf{v}$: a vector field on $\mathsf{N}$
- $\mathsf{u}$: this is not a vector field on M. This is constructed that the dual vector (one form) can apply to it. $\mathsf{u}(\mathsf{f}) (\mathsf{m}) = \mathsf{v}_{\mu}(\mathsf{f}) (\mathsf{n})$
- one-form over the map: $\omega^{\mu}(\mathsf{v}_{\mu}) (\mathsf{n}) = \omega(\mathsf{u}) (\mu(\mathsf{n}))$
- To extend this idea to k-forms, we carry each vector argument over the map

In [3]:
(clojure.repl/source form-field->form-field-over-map)

(defn form-field->form-field-over-map
  [mu:N->M]
  (fn [w-on-M]
    (letfn [(make-fake-vector-field [V-over-mu n]
              (vf/procedure->vector-field
               (fn [f]
                 (fn [_]
                   ((V-over-mu f) n)))
               `(~'make-fake-vector-field
                 ~(v/freeze V-over-mu))))
            (op [& vectors-over-map]
              (assert (= (count vectors-over-map)
                         (ff/get-rank w-on-M)))
              (fn [n]
                ((apply w-on-M
                        (map (fn [V-over-mu]
                               (make-fake-vector-field V-over-mu n))
                             vectors-over-map))
                 (mu:N->M n))))]
      (let [rank (ff/get-rank w-on-M)
            name `((~'form-field->form-field-over-map ~(v/freeze mu:N->M))
                   ~(v/freeze w-on-M))]
        (ff/procedure->nform-field op rank name)))))


nil

## 6.3 Basis Fields Over a Map

- $\mathsf{e}$: a tuple of basis vector fields
- $\mathsf{\tilde{e}}$: the tuple of basis one-forms that is dual to $\mathsf{e}$

We have $\mathsf{\tilde{e}}^i (\mathsf{e}_j) (\mathsf{m}) = \delta^i_j$

- $\mathsf{e}^{\mu}$: _basis vectors over the map_, which are particular cases of vectors over a map $\mathsf{e}^{\mu} (\mathsf{f}) = \mathsf{e} (\mathsf{f})\circ {\mu}$
- $\mathsf{\tilde{e}}_{\mu}$: _dual basis over the map_, which are particular cases of one-forms over the map, which satisfy $\mathsf{\tilde{e}}^i_{\mu} (\mathsf{e}_j^{\mu}) (\mathsf{n}) = \delta^i_j$

**Walking on a Sphere**
Let $\mu$ map the time line to the unit sphere

In [4]:
;; the map takes a point in the time line, map it to a real number
;; then map the real number to theta(t) and phi(t)
;; then the two coordinates are mapped back to the sphere
(def mu
    (compose (point S2-spherical)
             (up (literal-function 'theta)
                 (literal-function 'phi))
             (chart R1-rect)))

#'chp6/mu

- the basis over map is constructed from the basis on the sphere

In [5]:
(def S2-basis (coordinate-system->basis S2-spherical))
(def S2-basis-over-mu
    (basis->basis-over-map mu S2-basis))
(def h
    (literal-manifold-function 'h-spherical S2-spherical))

#'chp6/h

- The following shows the basis vectors over the map compute derivatives of the funciton $h$ evaluated on the path at the given time

In [6]:
(render
 (((basis->vector-basis S2-basis-over-mu) h)
  ((point R1-rect) 't0)))

"down(∂₀h-spherical(up(θ(t0), φ(t0))), ∂₁h-spherical(up(θ(t0), φ(t0))))"

In [7]:
(clojure.repl/source basis->vector-basis)

(defn basis->vector-basis
  "Extract the vector basis from the given basis object `b`."
  [b]
  {:pre [(basis? b)]}
  (:vector-basis b))


nil

In [8]:
S2-basis-over-mu

{:type :sicmutils.calculus.basis/basis, :dimension 2, :vector-basis (down ((vector-field->vector-field-over-map #function[clojure.lang.AFunction/1]) d:dx0) ((vector-field->vector-field-over-map #function[clojure.lang.AFunction/1]) d:dx1)), :oneform-basis (up ((form-field->form-field-over-map #function[clojure.lang.AFunction/1]) dx0) ((form-field->form-field-over-map #function[clojure.lang.AFunction/1]) dx1))}

- the dual basis over the map does the correct thing

In [9]:
(((basis->oneform-basis S2-basis-over-mu)
  (basis->vector-basis S2-basis-over-mu))
 ((point R1-rect) 't0))

(up (down 1 0) (down 0 1))

__Components of the Velocity__
- $\chi$: a tuple of coordinates on $\mathsf{M}$
- $\mathsf{X}_i$: basis vectors associates with $\chi$
- $\mathsf{dx}^i$: dual basis elements associates with $\chi$
- $\mathsf{X}_i^{\mu}$: basis vectors over the map $\mu$ associates with $\chi$
- $\mathsf{dx}^i_{\mu}$: dual basis elements over the map $\mu$ associates with $\chi$

The components of the velocity (rates of change of coordinates along the path $\mu$) are obtained by applying the dual basis over the map to the velocity
$$ v^i(t) = \mathsf{dx}^i_{\mu} (d\mu (\partial/\partial \mathsf{t})) (\mathsf{t}) $$

the coordinate velocities on a sphere are

In [12]:
(let-coordinates [t R1-rect]
 (((basis->oneform-basis S2-basis-over-mu)
   ((differential mu) d:dt))
  a-point))

(up ((D theta) t0) ((D phi) t0))

## 6.4 Pullbacks and Pushforwards

__For manifold functions__
- $\mu$: a map from manifold $\mathsf{N}$ to manifold $\mathsf{M}$
- pushforward of the function $\mathsf{g}$ on $\mathsf{N}$ over the map $\mu$: 
    - $\mu_*\mathsf{g} = \mathsf{g} \circ \mu^{-1}$ (we assume the inverse of the map $\mu$ exists)
- pullback of a function $\mathsf{f}$ on $\mathsf{M}$ over the map $\mu$: 
    - $\mu^*\mathsf{f} = \mathsf{f} \circ \mu$

In [10]:
(clojure.repl/source pushforward-function)

(defn pushforward-function [mu-inverse:M->N]
  (fn [f-on-N]
    (f/compose f-on-N mu-inverse:M->N)))


nil

In [11]:
(clojure.repl/source pullback-function)

(defn pullback-function
  [mu:N->M]
  (fn [f-on-M]
    (f/compose f-on-M mu:N->M)))


nil

Examples of pullback of manifold functions
- a vector field over the map constructed by restriction can be seen as the pullback of the function constructed by appication of the vector field to a function: $\mathsf{u}_{\mu}(\mathsf{f}) = \mathsf{u}(\mathsf{f})\circ {\mu} = \mu^*(\mathsf{u}(\mathsf{f}))$
- a vector field over the map constructed by a differential can be seen as the vector field applied to the pullback of the function: $d\mu(\mathsf{v})(\mathsf{f})(\mathsf{n}) = \mathsf{v}(\mathsf{f}\circ \mu) (\mathsf{n}) = \mathsf{v}(\mu^*\mathsf{f} ) (\mathsf{n})$

__For vector fields__
- $\mu$: a map from manifold $\mathsf{N}$ to manifold $\mathsf{M}$
- pushforward of a vector field $\mathsf{v}$ defined on $\mathsf{N}$ over the map $\mu$: 
    - $\mu_*\mathsf{v}(\mathsf{f}) (\mathsf{m}) = (\mathsf{v} (\mathsf{f} \circ \mu)) (\mu^{-1} (\mathsf{m})) = \mu_*(\mathsf{v} (\mu^*\mathsf{f}))$
    - The RHS is expressed in pushback and pushforward of manifold functions
    - for a map from time to some configuration manifold. The pushforward of a vector field is the velocity measured at a point on the trajectory in the configuration manifold. However, the pushforward is not defined at any point where crosssing occurs. By contrast, the differential of the map applied to the (source) vector field gives us the velocity vector at each moment in time and is always defined. 
- pullback of a vector field $\mathsf{u}$ on $\mathsf{M}$ over the map $\mu$: 
    - $\mu^*\mathsf{u}(\mathsf{g}) (\mathsf{n}) = (\mathsf{u} (\mathsf{g} \circ \mu^{-1})) (\mu (\mathsf{n})) = \mu^*(\mathsf{u} (\mu_*\mathsf{g}))$

In [12]:
(clojure.repl/source pushforward-vector)

(defn pushforward-vector
  [mu:N->M mu-inverse:M->N]
  (fn [v-on-N]
    (let [op (fn [f]
               (f/compose (v-on-N (f/compose f mu:N->M))
                          mu-inverse:M->N))
          name `((~'pushforward ~(v/freeze mu:N->M))
                 ~(v/freeze v-on-N))]
      (vf/procedure->vector-field op name))))


nil

In [13]:
(clojure.repl/source pullback-vector-field)

(defn pullback-vector-field
  [mu:N->M mu-inverse:M->N]
  (pushforward-vector mu-inverse:M->N mu:N->M))


nil

__Integral Curves__
- $\phi_t^{\mathsf{w}}(\mathsf{m})$: the integral curve of $\mathsf{w}$ evolved for time $t$ as a function of the initial manifold point $\mathsf{m}$, it is a map of the manifold onto itself.
    - The evolution of the function $\mathsf{f}$ along an integral curve can be written in terms of the pullback over $\phi_t^{\mathsf{w}}$: $(E_{t, \mathsf{w}}) (\mathsf{m}) = \mathsf{f} (\phi_t^{\mathsf{w}}(\mathsf{m})) = ((\phi_t^{\mathsf{w}})^*\mathsf{f}) (\mathsf{m})$. 
- Since the map induced by integral curves is always invertible (uniqueness of the solutions of the initial-value problem fro ODE), we can push a vector field $\mathsf{v}$ forward over the map generated by an integral curve of a vector field $\mathsf{w}$

$$ (\phi_t^{\mathsf{w}})_*\mathsf{v}(\mathsf{f}) (\mathsf{m}) = (\mathsf{v} (\mathsf{f} \circ \phi_t^{\mathsf{w}})) (\phi_{-t}^{\mathsf{w}} (\mathsf{m}))$$
This is implemented as

In [14]:
(clojure.repl/source pushforward-vector)

(defn pushforward-vector
  [mu:N->M mu-inverse:M->N]
  (fn [v-on-N]
    (let [op (fn [f]
               (f/compose (v-on-N (f/compose f mu:N->M))
                          mu-inverse:M->N))
          name `((~'pushforward ~(v/freeze mu:N->M))
                 ~(v/freeze v-on-N))]
      (vf/procedure->vector-field op name))))


nil

__For Form Field__
- $\mu$: a map from manifold $\mathsf{N}$ to manifold $\mathsf{M}$
- Pushforward of a one-form field $\omega$ defined on $\mathsf{N}$ over a map $\mu$
    - $\mu_* \omega (\mathsf{u}) = \mu_* (\omega (\mu^* \mathsf{u}))$
    - it is rarely useful
- Pullback of a one-form field $\omega$ defined on $\mathsf{M}$ over a map $\mu$
    - $\mu^* \omega (\mathsf{v}) = \mu^* (\omega (\mu_*\mathsf{v}))$
    - The above is accurate, but not effective, because computing the pushforward of a vector field requires the inverse of the map $\mu$. However, we can avoid needing the inverse by leveraging the differential of a map.
    - $\mu^* \omega (\mathsf{v}) = \omega^{\mu} (d\mu(\mathsf{v})))$
    - This equation is valid because of the following reasoning. Imagine the map $\mu$ is invertible. Then for each integral curve in $\mathsf{N}$ generated by a vector field $\mathsf{v}$, its image on $\mathsf{M}$ is an integral curve generated by $\mu_*\mathsf{v}$. The pushforward of vector field exist since the map is invertible. Now, $\mu^* \omega (\mathsf{v}) (\mathsf{n}) = (\omega (\mu_*\mathsf{v})) (\mathsf{m}) $. The pushforward of $\mathsf{v}$ can also be viewed as transporting $\mathsf{v}$ from $\mathsf{N}$ to $\mathsf{M}$. This is similar to _differential_ of the map $\mu$ acting on $\mathsf{v}$! But the differential of the map is a vector field over the map, which means it is evaluated at $\mathsf{N}$, by definition of the one-form field over a map, it is not hard to see the above equation holds. It holds even if the the map is not invertible. In this case, we can visually think of calculating the pullback of a one-form field restricted to an intergral curve. 
- pullback of a _k_-form.
    - $\mu^* \omega (\mathsf{u}, \mathsf{v}, ...) (\mathsf{n}) = \omega (\mu_*\mathsf{u}, \mu_*\mathsf{v}, ...) (\mu(\mathsf{n}))$

In [15]:
(clojure.repl/source pullback-form)

(defn pullback-form
  "Returns a function which will pull a form back across a map (without needing
  its inverse)"
  [mu:N->M]
  (fn [omega-on-M]
    (let [k (ff/get-rank omega-on-M)]
      (if (zero? k)
        ((pullback-function mu:N->M) omega-on-M)
        (let [op (fn [& vectors-on-N]
                   (apply ((form-field->form-field-over-map mu:N->M) omega-on-M)
                          (map (differential mu:N->M)
                               vectors-on-N)))
              name `((~'pullback ~(v/freeze mu:N->M))
                     ~(v/freeze omega-on-M))]
          (ff/procedure->nform-field op k name))))))


nil

__Properties of Pullback__
- distributes through addition and through wedge product
$$
\begin{align}
    \mu^*(\theta + \phi) &= \mu^*\theta + \mu^*\phi \\
    \mu^*(\theta \wedge \phi) &= \mu^*\theta \wedge \mu^*\phi
\end{align}
$$
- commutes with the exterior derivative (here $\theta$ is a manifold function or a k-form field)
$$ \mathsf{d}(\mu^*\theta) = \mu^*(\mathsf{d}\theta)$$

In [18]:
;; a map from the rectangular plane to rectangular 3-space
(def mu
    (literal-manifold-map 'MU R2-rect R3-rect))

#'chp6/mu

In [19]:
;; verify statement with manifold function 
(def f (literal-manifold-function 'f-rect R3-rect))
(def X (literal-vector-field 'X-rect R2-rect))
(render
 (((- ((pullback mu) (d f)) (d ((pullback mu) f))) X)
  ((point R2-rect) (up 'x0 'y0))))

"0"

In [21]:
;; verify statement with a form field
(def theta (literal-oneform-field 'THETA R3-rect))
(def Y (literal-vector-field 'Y-rect R2-rect))
(render
 (((- ((pullback mu) (d theta)) (d ((pullback mu) theta))) X Y)
  ((point R2-rect) (up 'x0 'y0))))

"0"

### Ex 6.1 Velocities on a Globe
__a.__ Suppose that a vehicle is traveling east on the Earth at a given rate of change of longitude. That is the actual ground speed of the vehicle?

In [53]:
(defn v-colatitude
    [t]
    'init-colatitude)
(defn v-longtitude
    [t]
    (* t 'v-long))
(def mu-a
    (compose (point S2-spherical)
             (up v-colatitude ;; colatitude
                 v-longtitude)  ;; longtitude
             (chart R1-rect)))

#'chp6/mu-a

In [54]:
(def S2-basis (coordinate-system->basis S2-spherical))
(def S2-basis-over-mu-a
    (basis->basis-over-map mu-a S2-basis))

#'chp6/S2-basis-over-mu-a

- since we haven't defined a metric yet, the following speed is the speed on the coordinate plane.

In [61]:
(let-coordinates
 [t R1-rect]
 (((basis->oneform-basis S2-basis-over-mu-a)
   ((differential mu-a) d:dt))
   ((point R1-rect) 't0)))

(up 0 v-long)

__b__. Sterographic projection is useful for navigation because it is conformal (it preserves angles). For the situation of part __a__, what is the spped measured on a stereographic map? 

In [56]:
;; map from R1 to R2
(def mu-b
    (compose (point R2-rect)
             (chart S2-Riemann)
             (point S2-spherical)
             (up v-colatitude ;; colatitude
                 v-longtitude)  ;; longtitude
             (chart R1-rect)))

#'chp6/mu-b

In [57]:
(def R2-basis (coordinate-system->basis R2-rect))
(def R2-basis-over-mu-b
    (basis->basis-over-map mu-b R2-basis))

#'chp6/R2-basis-over-mu-b

In [60]:
(let-coordinates
 [t R1-rect]
 (render
  (((basis->oneform-basis R2-basis-over-mu-b)
    ((differential mu-b) d:dt))
    ((point R1-rect) 't0))))

"up((v-long sin(init-colatitude) sin(t0 v-long)) / (cos(init-colatitude) + -1), - v-long sin(init-colatitude) cos(t0 v-long) / (cos(init-colatitude) + -1))"